# 텍스트 분류 (뉴스그룹 분류)

Raw Text Data --> technology, sports, fashin... 등으로 classification을 해볼 것이다.  
Data는 사이킷런의 fetch_20newsgroups()를 이용한다.

In [1]:
from sklearn.datasets import fetch_20newsgroups
news_data= fetch_20newsgroups(subset='all',random_state=156)

In [3]:
print(news_data.keys())

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])


fetch_20newsgroups()의 filenames라는 key는 API가 인터넷에서 내려받아 로컬 컴퓨터에 저장하는 디렉토리와 파일명을 지칭한다.  
다음으로 target 클래스가 어떻게 구성돼 있는지 확인해보자

In [13]:
import pandas as pd
print('target 클래스의 값과 분포도 \n',pd.Series(news_data.target).value_counts().sort_index())
print('target 클래스의 이름들 \n',news_data.target_names)

target 클래스의 값과 분포도 
 0     799
1     973
2     985
3     982
4     963
5     988
6     975
7     990
8     996
9     994
10    999
11    991
12    984
13    990
14    987
15    997
16    910
17    940
18    775
19    628
dtype: int64
target 클래스의 이름들 
 ['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


target 클래스는 0~19까지 20개로 구성되어 있다. 또한 target:0 --> alt.ahteism, target:1 --> comp.graphics ... 등으로 되어 있다.  
개별 데이터는 어떻게 구성되어 있을까?

In [23]:
print(news_data.data[0])

From: egreen@east.sun.com (Ed Green - Pixel Cruncher)
Subject: Re: Observation re: helmets
Organization: Sun Microsystems, RTP, NC
Lines: 21
Distribution: world
Reply-To: egreen@east.sun.com
NNTP-Posting-Host: laser.east.sun.com

In article 211353@mavenry.altcit.eskimo.com, maven@mavenry.altcit.eskimo.com (Norman Hamer) writes:
> 
> The question for the day is re: passenger helmets, if you don't know for 
>certain who's gonna ride with you (like say you meet them at a .... church 
>meeting, yeah, that's the ticket)... What are some guidelines? Should I just 
>pick up another shoei in my size to have a backup helmet (XL), or should I 
>maybe get an inexpensive one of a smaller size to accomodate my likely 
>passenger? 

If your primary concern is protecting the passenger in the event of a
crash, have him or her fitted for a helmet that is their size.  If your
primary concern is complying with stupid helmet laws, carry a real big
spare (you can put a big or small head in a big helmet, bu

데이터는 다음과 같이 구성되어 있으며 기사 내용뿐 아니라 뉴스그룹 제목, 작성자, 소속, 이메일 등 다양한 정보 갖고 있다. 이 중에서 내용을 제외하고 제목 등의 다른 정보는 제거하겠다. 왜냐하면 제목과 소속, 이메일 주소 등의 header, footer 정보들은 뉴스그룹 분류의 Target 클래스 값과 유사한 데이터를 갖고 있는 경우가 많기 때문이다.

따라서 우리는 순수한 텍스트만으로 구성된 기사 내용으로 어떤 뉴스그룹에 속하는지 분류할 것이다. remove 파라미터를 이용하면 뉴스그룹 기사의 header, footer 등을 제거할 수 있다. 또한 fetch_20newsgroups()는 subset 파라미터를 이용해 train,test 세트를 분리해 내려받을 수 있다.

In [30]:
from sklearn.datasets import fetch_20newsgroups
# subset=train으로 학습용 데이터만 추출 remove=('headers','footers','quotes')
train_news=fetch_20newsgroups(subset='train',remove=('headers','footers','quotes'),random_state=156)

X_train=train_news.data
y_train=train_news.target

# subset=test으로 테스트 데이터만 추출 remove=('headers','footers','quotes')
test_news=fetch_20newsgroups(subset='test',remove=('headers','footers','quotes'),random_state=156)

X_test=test_news.data
y_test=test_news.target

print('학습 데이터 크기 {0},테스트 데이터 크기 {1}'.format(len(train_news.data),len(test_news.data)))

학습 데이터 크기 11314,테스트 데이터 크기 7532


## 피처 벡터화 변환과 머신러닝 모델 학습/예측/평가

CountVectorizer를 이용해 학습 데이터의 텍스트를 피처 벡터화할것이다. 여기서 유의해야 할 점이 있다.  

test 데이터에서 CountVectorizer를 적용할 때는 반드시 train 데이터를 이용해 fit()이 수행된 CountVecotorizer 객체를 이용해 test 데이터를 transform해야 한다는 것이다. 그래야만 학습 시 설정된 CountVectorizer의 피처 개수와 test 데이터를 CountVectorizer로 변환할 피처 개수가 같아진다는 것이다. 따라서 test 데이터의 피처 벡터화는 train 데이터에 사용된 CountVectorizer 객체 변수인 cnt_vect.transform()을 이용해 변환한다.

또한 test 데이터의 피처 벡터화 시 fit_transform()을 사용하면 안 된다는 점도 유의해야한다.  
CountVectorizer.fit_transform(test데이터)를 하게 된다면 test 데이터 기반으로 다시 CountVectorizer가 fit()을 수행하고 transform()하기 때문에 학습 시 사용된 피처 개수와 예측 시 사용할 피처 개수가 달라지게 된다.

### Count 기반

In [60]:
import sklearn.feature_extraction.text

In [61]:
from sklearn.feature_extraction.text import CountVectorizer

In [55]:
# Count Vectorization으로 피처 벡터화 변환 수행
cnt_vect=CountVectorizer()
cnt_vect.fit(X_train,y_train)
X_train_cnt_vect=cnt_vect.transform(X_train)

# 학습 데이터로 fit()된 CountVectorizer를 이용해 테스트 데이터를 피처 벡터화 변환 수행.
X_test_cnt_vect=cnt_vect.transform(X_test)

print('학습 데이터 텍스트의 CountVectorizer Shape:',X_test_cnt_vect.shape)

학습 데이터 텍스트의 CountVectorizer Shape: (7532, 101631)


In [56]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# LogisticRegression을 이용해 학습/예측/평가를 수행.
lr_clf=LogisticRegression()
lr_clf.fit(X_train_cnt_vect,y_train)
pred= lr_clf.predict(X_test_cnt_vect)

print('CountVectorized Logistic Regression의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test,pred)))

C:\Users\kiwon\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\kiwon\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


CountVectorized Logistic Regression의 예측 정확도는 0.617


### TF-IDF 기반

In [63]:
import sklearn.feature_extraction.text

In [64]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [66]:
# TF-IDF 벡터화를 적용해 학습 데이터 세트와 테스트 데이터 세트 변환
tfidf_vect=TfidfVectorizer()
tfidf_vect.fit(X_train)
X_train_tfidf_vect=tfidf_vect.transform(X_train)
X_test_tfidf_vect=tfidf_vect.transform(X_test)

# LogisticRegression을 이용해 학습/예측/평가 수행
lr_clf=LogisticRegression()
lr_clf.fit(X_train_tfidf_vect,y_train)
pred=lr_clf.predict(X_test_tfidf_vect)
print("TF-IDF Logistic Regression의 예측 정확도는 {0:.3f}".format(accuracy_score(y_test,pred)))

C:\Users\kiwon\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\kiwon\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


TF-IDF Logistic Regression의 예측 정확도는 0.678


예측 결과를 보면 TF-IDF가 단순 카운트 기반보다 높은 예측 정확도를 보여준다. 여기서 성능을 더욱 높이기 위해 파라미터 조정을 해보겠다.  
TfidfVectorizer 클래스의 스톱 워드를 기존'None'에서 'english'로 변경하고, ngram_range는 기존(1,1)에서 (1,2)로, max_df=300으로 변경한 뒤 다시 예측 성능을 측정해 보겠다.

In [69]:
# stop words 필터링 추가 & ngram을 기본 (1,1)에서 (1,2)
tfidf_vect=TfidfVectorizer(stop_words='english',ngram_range=(1,2),max_df=300)
tfidf_vect.fit(X_train)
X_train_tfidf_vect=tfidf_vect.transform(X_train)
X_test_tfidf_vect=tfidf_vect.transform(X_test)

lr_clf=LogisticRegression()
lr_clf.fit(X_train_tfidf_vect,y_train)

C:\Users\kiwon\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\kiwon\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [71]:
pred=lr_clf.predict(X_test_tfidf_vect)
print('TF-IDF Vectorized Logistic Regression 의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test,pred)))

TF-IDF Vectorized Logistic Regression 의 예측 정확도는 0.690


0.678-->0.690으로 상승한 것을 확인할 수 있다.  
다음은 GridSearchCV를 이용해 로지스틱 회귀의 하이퍼 파라미터 최적화를 수행해 볼 것이다. 로지스틱 회귀의 C 파라미터만 변경하면서 최적의 C값을 찾은 뒤 이 C값으로 학습된 모델에서 테스트 데이터로 예측해 성능을 평가하겠다.

In [73]:
from sklearn.model_selection import GridSearchCV

# 최적 C 값 도출 튜닝 수행. CV는 3 fold set 
params={'C':[0.01,0.1,1,5,10]}
grid_cv_lr=GridSearchCV(lr_clf,param_grid=params,cv=3,scoring='accuracy',verbose=1)
grid_cv_lr.fit(X_train_tfidf_vect,y_train)
print('Logistic Regression best C parameter:',grid_cv_lr.best_params_)

# 최적 C 값으로 학습된 grid_cv로 예측 및 정확성 평가
pred= grid_cv_lr.predict(X_test_tfidf_vect)
print('TF-IDF Vectorized Logistic Regression의 예측 정확도는:{0:3f}'.format(accuracy_score(y_test,pred)))

Fitting 3 folds for each of 5 candidates, totalling 15 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
C:\Users\kiwon\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\kiwon\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:  5.4min finished


Logistic Regression best C parameter: {'C': 10}
TF-IDF Vectorized Logistic Regression의 예측 정확도는:0.703930
